<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-&amp;-Inits" data-toc-modified-id="Imports-&amp;-Inits-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports &amp; Inits</a></span></li><li><span><a href="#Data-&amp;-Model" data-toc-modified-id="Data-&amp;-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data &amp; Model</a></span></li></ul></div>

# AG News Classifier with ConvNet
Classifier to classify news titles into categories.

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pdb
import pandas as pd
import numpy as np
import torch

from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader
from pathlib import Path

from ignite.engine import Events, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [ ]:
from consts import consts
from ag.data import NewsDataset, DataContainer
consts

## Data & Model

In [ ]:
df = pd.read_csv(consts.proc_dataset_csv)
df.head()

In [ ]:
dc = DataContainer(df, NewsDataset, consts.vectorizer_file, consts.batch_size)

In [ ]:
class NewsClassifier(nn.Module):
  def __init__(self, emb_sz, vocab_size, n_channels, hidden_dim, n_classes, dropout_p,
               pretrained=None, padding_idx=0):
    super(NewsClassifier, self).__init__()
    
    if pretrained:
      pass
    else:
      self.emb = nn.Embedding(vocab_size, emb_sz, padding_idx)
      
    self.convnet = nn.Sequential(
      nn.Conv1d(in_channels=emb_sz, out_channels=n_channels, kernel_size=3),
      nn.ELU(),
      nn.Conv1d(in_channels=n_channels, out_channels=n_channels, kernel_size=3, stride=2),
      nn.ELU(),
      nn.Conv1d(in_channels=n_channels, out_channels=n_channels, kernel_size=3, stride=2),
      nn.ELU(),
      nn.Conv1d(in_channels=n_channels, out_channels=n_channels, kernel_size=3),
      nn.ELU()
    )
    
    self.dropout = nn.Dropout(p=dropout_p)
    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(in_features=n_channels, out_features=hidden_dim)
    self.fc2 = nn.Linear(in_features=hidden_dim, out_features=n_classes)
    self.softmax = nn.Softmax(dim=1)
    
  def forward(self, x_in, apply_softmax=False):
    # embed and permute so features are channels
    x_emb = self.emb(x_in).permute(0,2,1)
    features = self.convnet(x_emb)
    
    # average and remove extra dimension
    remaining_size = features.size(dim=2)
    features = F.avg_pool1d(features, remaining_size).squeeze(dim=2)
    features = self.dropout(features)
    
    # mlp classifier
    hidden_vector = self.fc1(features)
    hidden_vector = self.dropout(hidden_vector)
    hidden_vector = self.relu(hidden_vector)
    prediction_vector = self.fc2(hidden_vector)
    
    if apply_softmax:
      prediction_vector = self.softmax(prediction_vector)
      
    return prediction_vector

In [ ]:
classifier = NewsClassifier(consts.embedding_size, dc.vocabulary_size, consts.n_channels, consts.hidden_dim,
                            dc.n_classes, consts.dropout_p)

In [ ]:
classifier

In [ ]:
itr = iter(dc.train_dl)

In [ ]:
x,y = next(itr)